In [1]:
import numpy as np
import pandas as pd
import altair as alt

In [2]:
# load in data
opioid_deaths = pd.read_parquet("/Users/erikafox/Desktop/pds2021-opioids-pds6/10_modified_data/vital_pop.parquet", engine='fastparquet')
opioid_buyers = pd.read_parquet("/Users/erikafox/Desktop/pds2021-opioids-pds6/10_modified_data/opi_pop.parquet", engine='fastparquet')

In [3]:
opioid_buyers.sample(10)

,BUYER_STATE,BUYER_COUNTY,fips,year,MME,Population,MME per cap,treatment
index,,,,,,,,
2140,GA,OCONEE,13219,2012,6.756682e+06,33514.0,201.607742,0
3340,LA,ST. HELENA,22091,2013,1.032689e+06,10771.0,95.876807,0
4592,MT,SHERIDAN,30091,2012,7.118908e+05,3550.0,200.532613,0
6345,SC,YORK,45091,2012,5.644417e+07,234059.0,241.153604,0
8558,WA,KLICKITAT,53039,2008,2.560843e+06,20135.0,127.183671,0
7895,TX,REEVES,48389,2013,3.688686e+06,14494.0,254.497477,0
7400,TX,KERR,48265,2007,1.298518e+07,48352.0,268.555204,0
6528,TX,BREWSTER,48043,2012,2.427151e+06,9259.0,262.139617,0
1542,GA,CRISP,13081,2011,8.737890e+06,23753.0,367.864695,0


In [4]:
opioid_deaths.sample(10)

,STNAME,CTYNAME,fips,Year,Deaths,vital deaths per cap,treatment
index,,,,,,,
1933,Colorado,Denver County,08031,2009,160,0.000272,0
6074,Virginia,Russell County,51167,2005,10,0.000351,0
823,Kansas,Butler County,20015,2013,11,0.000168,0
5683,Maryland,Prince George's County,24033,2006,28,0.000033,0
6888,Tennessee,Sullivan County,47163,2006,24,0.000156,0
514,Texas,Bexar County,48029,2010,186,0.000108,1
2552,Michigan,Genesee County,26049,2009,77,0.000180,0
3711,Illinois,LaSalle County,17099,2015,25,0.000225,0
5235,New York,Ontario County,36069,2014,11,0.000101,0


In [5]:
# Creating florida buyers df

opioid_buyers_fl=opioid_buyers[opioid_buyers["BUYER_STATE"]=="FL"]


opioid_buyers_fl["policy_years"]= opioid_buyers_fl["year"]-2010


fl_opi_before=opioid_buyers_fl[opioid_buyers_fl["policy_years"]<0]
fl_opi_after=opioid_buyers_fl[opioid_buyers_fl["policy_years"]>=0]



/var/folders/5b/sjkc4b8n0hs41zd03hxr66100000gn/T/ipykernel_44424/4177260778.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opioid_buyers_fl["policy_years"]= opioid_buyers_fl["year"]-2010


In [6]:
# florida buyers pre/post plot

before=alt.Chart(fl_opi_before,title="Pre-Post Policy: florida, Opioids per cap").encode(x="policy_years", y="MME per cap")
before_line=before.transform_regression("policy_years","MME per cap").mark_line() 
after=alt.Chart(fl_opi_after).encode(x="policy_years", y="MME per cap")
after_line=after.transform_regression("policy_years","MME per cap").mark_line()
fl_chart_opioid=before_line+after_line
fl_chart_opioid


alt.LayerChart(...)

In [7]:
# Creating florida buyers df

opioid_deaths_fl=opioid_deaths[opioid_deaths["STNAME"]=="Florida"]

opioid_deaths_fl["policy_years"]= opioid_deaths_fl["Year"]-2010


fl_death_before=opioid_deaths_fl[opioid_deaths_fl["policy_years"]<0]
fl_death_after=opioid_deaths_fl[opioid_deaths_fl["policy_years"]>=0]


/var/folders/5b/sjkc4b8n0hs41zd03hxr66100000gn/T/ipykernel_44424/2155771455.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opioid_deaths_fl["policy_years"]= opioid_deaths_fl["Year"]-2010


In [8]:
# florida deaths pre/post plot

before=alt.Chart(fl_death_before,title="Pre-Post Policy: Florida, Overdose Deaths per cap").encode(x="policy_years", y="vital deaths per cap")
before_line=before.transform_regression("policy_years","vital deaths per cap").mark_line() 
after=alt.Chart(fl_death_after).encode(x="policy_years", y="vital deaths per cap")
after_line=after.transform_regression("policy_years","vital deaths per cap").mark_line()
fl_chart_death=before_line+after_line
fl_chart_death

alt.LayerChart(...)

In [9]:
# Creating sc dataframes
opioid_buyers_sc=opioid_buyers[opioid_buyers["BUYER_STATE"]=="SC"]
opioid_buyers_sc.sample(10)
opioid_buyers_sc["policy_years"]= opioid_buyers_sc["year"]-2010

sc_opi_before=opioid_buyers_sc[opioid_buyers_sc["policy_years"]<0]
sc_opi_after=opioid_buyers_sc[opioid_buyers_sc["policy_years"]>=0]


opioid_deaths_sc=opioid_deaths[opioid_deaths["STNAME"]=="South Carolina"]
opioid_deaths_sc["policy_years"]= opioid_deaths_sc["Year"]-2010


sc_death_before=opioid_deaths_sc[opioid_deaths_sc["policy_years"]<0]
sc_death_after=opioid_deaths_sc[opioid_deaths_sc["policy_years"]>=0]


/var/folders/5b/sjkc4b8n0hs41zd03hxr66100000gn/T/ipykernel_44424/390277016.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opioid_buyers_sc["policy_years"]= opioid_buyers_sc["year"]-2010
/var/folders/5b/sjkc4b8n0hs41zd03hxr66100000gn/T/ipykernel_44424/390277016.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opioid_deaths_sc["policy_years"]= opioid_deaths_sc["Year"]-2010


In [10]:
# sc buyers pre/post plot

before=alt.Chart(sc_opi_before,title="Pre-Post Policy: South Carolina, Opioids per cap").encode(x="policy_years", y="MME per cap")
before_line=before.transform_regression("policy_years","MME per cap").mark_line(color="#FFAA00") 
after=alt.Chart(sc_opi_after).encode(x="policy_years", y="MME per cap")
after_line=after.transform_regression("policy_years","MME per cap").mark_line(color="#FFAA00")
sc_chart_opioid=before_line+after_line
sc_chart_opioid

alt.LayerChart(...)

In [11]:
# South Carolina deaths pre/post plot

before=alt.Chart(sc_death_before,title="Pre-Post Policy: South Carolina, Overdose Deaths per cap").encode(x="policy_years", y="vital deaths per cap")
before_line=before.transform_regression("policy_years","vital deaths per cap").mark_line(color="#FFAA00") 
after=alt.Chart(sc_death_after).encode(x="policy_years", y="vital deaths per cap")
after_line=after.transform_regression("policy_years","vital deaths per cap").mark_line(color="#FFAA00")
sc_chart_death=before_line+after_line
sc_chart_death

alt.LayerChart(...)

In [12]:
#Difference in difference plot, deaths, FL vs SC
#how do we change the title???
sc_chart_death+fl_chart_death

alt.LayerChart(...)

In [13]:
#Difference in difference plot, opioids, FL vs SC
#how do we change the title???
sc_chart_opioid+fl_chart_opioid

alt.LayerChart(...)